In [6]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 2: Bathroom Security ---</h2><p>You arrive at <em>Easter Bunny Headquarters</em> under cover of darkness. However, you left in such a rush that you forgot to use the bathroom! Fancy office buildings like this one usually have keypad locks on their bathrooms, so you search the front desk for the code.</p>
<p>"In order to improve security," the document you find says, "bathroom codes will no longer be written down.  Instead, please memorize and follow the procedure below to access the bathrooms."</p>
<p>The document goes on to explain that each button to be pressed can be found by starting on the previous button and moving to adjacent buttons on the keypad: <code>U</code> moves up, <code>D</code> moves down, <code>L</code> moves left, and <code>R</code> moves right. Each line of instructions corresponds to one button, starting at the previous button (or, for the first line, <em>the "5" button</em>); press whatever button you're on at the end of each line. If a move doesn't lead to a button, ignore it.</p>
<p>You can't hold it much longer, so you decide to figure out the code as you walk to the bathroom. You picture a keypad like this:</p>
<pre><code>1 2 3
4 5 6
7 8 9
</code></pre>
<p>Suppose your instructions are:</p>
<pre><code>ULL
RRDDD
LURDL
UUUUD
</code></pre>
<ul>
<li>You start at "5" and move up (to "2"), left (to "1"), and left (you can't, and stay on "1"), so the first button is <code>1</code>.</li>
<li>Starting from the previous button ("1"), you move right twice (to "3") and then down three times (stopping at "9" after two moves and ignoring the third), ending up with <code>9</code>.</li>
<li>Continuing from "9", you move left, up, right, down, and left, ending with <code>8</code>.</li>
<li>Finally, you move up four times (stopping at "2"), then down once, ending with <code>5</code>.</li>
</ul>
<p>So, in this example, the bathroom code is <code>1985</code>.</p>
<p>Your puzzle input is the instructions from the document you found at the front desk. What is the <em>bathroom code</em>?</p>
</article>


In [7]:
instructions = """
ULL
RRDDD
LURDL
UUUUD
"""


def decode(instructions: str) -> int:
    keypad = [[s + d for d in range(3)] for s in range(1, 10, 3)]
    code = 0
    row, col = 1, 1
    for line in instructions.strip().splitlines():
        for inst in line.strip():
            if inst == "U" and row - 1 > -1:  # U moves up,
                row -= 1
            elif inst == "D" and row + 1 < 3:  # D moves down,
                row += 1
            elif inst == "L" and col - 1 > -1:  # L moves left,
                col -= 1
            elif inst == "R" and col + 1 < 3:  # and R moves right
                col += 1
        code = 10 * code + keypad[row][col]
    return code


assert decode(instructions) == 1985

In [8]:
with open("../input/day2.txt") as f:
    print(decode(f.read()))

56855


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>56855</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You finally arrive at the bathroom (it's a several minute walk from the lobby so visitors can behold the many fancy conference rooms and water coolers on this floor) and go to punch in the code.  Much to your bladder's dismay, the keypad is not at all like you imagined it.  Instead, you are confronted with the result of hundreds of man-hours of <span title="User Group 143 found a diamond shape to be the most environmentally friendly.">bathroom-keypad-design meetings:</span></p>
<pre><code>    1
  2 3 4
5 6 7 8 9
  A B C
    D
</code></pre>
<p>You still start at "5" and stop when you're at an edge, but given the same instructions as above, the outcome is very different:</p>
<ul>
<li>You start at "5" and don't move at all (up and left are both edges), ending at <code>5</code>.</li>
<li>Continuing from "5", you move right twice and down three times (through "6", "7", "B", "D", "D"), ending at <code>D</code>.</li>
<li>Then, from "D", you move five more times (through "D", "B", "C", "C", "B"), ending at <code>B</code>.</li>
<li>Finally, after five more moves, you end at <code>3</code>.</li>
</ul>
<p>So, given the actual keypad layout, the code would be <code>5DB3</code>.</p>
<p>Using the same instructions in your puzzle input, what is the correct <em>bathroom code</em>?</p>
</article>

</main>


In [9]:
def decode_II(instructions: str) -> int:
    # fmt: off
    keypad = [
        [  0,   0, 0x1,   0,   0],  #     1
        [  0, 0x2, 0x3, 0x4,   0],  #   2 3 4
        [0x5, 0x6, 0x7, 0x8, 0x9],  # 5 6 7 8 9
        [  0, 0xa, 0xb, 0xc,   0],  #   A B C
        [  0,   0, 0xd,   0,   0],  #     D
    ]
    # fmt: on
    code = 0
    row, col = 2, 0
    for line in instructions.strip().splitlines():
        for inst in line.strip():
            if (
                inst == "U" and row - 1 > -1 and keypad[row - 1][col] != 0
            ):  # U moves up,
                row -= 1
            elif (
                inst == "D" and row + 1 < 5 and keypad[row + 1][col] != 0
            ):  # D moves down,
                row += 1
            elif (
                inst == "L" and col - 1 > -1 and keypad[row][col - 1] != 0
            ):  # L moves left,
                col -= 1
            elif (
                inst == "R" and col + 1 < 5 and keypad[row][col + 1] != 0
            ):  # and R moves right
                col += 1
        code = 16 * code + keypad[row][col]
    return code


assert decode_II(instructions) == 0x5DB3

print(f"{decode_II(instructions):02x}".upper())

5DB3


In [10]:
with open("../input/day2.txt") as f:
    print(f"{decode_II(f.read()):02x}".upper())

B3C27


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>B3C27</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="2/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Bathroom+Security%22+%2D+Day+2+%2D+Advent+of+Code+2016&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F2&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Bathroom+Security%22+%2D+Day+2+%2D+Advent+of+Code+2016+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F2';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
